# **SCHEMA ENFORCEMENT**

In [0]:
data = [(1,100,'aa',1),(2,200,'bb',1),(3,300,'cc',1),(4,400,'dd',1)]

df = spark.createDataFrame(data, ['id','salary','name','sus_col'])

display(df)

id,salary,name,sus_col
1,100,aa,1
2,200,bb,1
3,300,cc,1
4,400,dd,1


In [0]:
df.write.format("delta")\
        .mode("append")\
        .save("/Volumes/deltalakeansh/default/deltavol/demosink/")

---------------------------------------------------------------------------
AnalysisException Traceback (most recent call last)
File , line 3
 1 df.write.format("delta")\
 2 .mode("append")\
----> 3 .save("/Volumes/deltalakeansh/default/deltavol/demosink/")

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/readwriter.py:679, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
 677 self.format(format)
 678 self._write.path = path
--> 679 _, _, ei = self._spark.client.execute_command(
 680 self._write.command(self._spark.client), self._write.observations
 681 )
 682 self._callback(ei)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1297, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
 1295 req.user_context.user_id = self._user_id
 1296 req.plan.command.CopyFrom(command)
-> 1297 data, _, metrics, observed_metrics, properties = self._execute_and_fetch(
 1298 req, observations or {}, extra_request_metadata
 1299 )
 1300 # Create a query execution object.
 1301 ei = ExecutionInfo(metrics, observed_metrics)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1755, in SparkConnectClient._execute_and_fetch(self, req, observations, extra_request_metadata, self_destruct)
 1752 properties: Dict[str, Any] = {}
 1754 with Progress(handlers=self._progress_handlers, operation_id=req.operation_id) as progress:
-> 1755 for response in self._execute_and_fetch_as_iterator(
 1756 req, observations, extra_request_metadata or [], progress=progress
 1757 ):
 1758 if isinstance(response, StructType):
 1759 schema = response

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1731, in SparkConnectClient._execute_and_fetch_as_iterator(self, req, observations, extra_request_metadata, progress)
 1729 raise kb
 1730 except Exception as error:
-> 1731 self._handle_error(error)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:2047, in SparkConnectClient._handle_error(self, error)
 2045 self.thread_local.inside_error_handling = True
 2046 if isinstance(error, grpc.RpcError):
-> 2047 self._handle_rpc_error(error)
 2048 elif isinstance(error, ValueError):
 2049 if "Cannot invoke RPC" in str(error) and "closed" in str(error):

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:2149, in SparkConnectClient._handle_rpc_error(self, rpc_error)
 2134 raise Exception(
 2135 "Python versions in the Spark Connect client and server are different. "
 2136 "To execute user-defined functions, client and server should have the "
 (...)
 2145 "https://docs.databricks.com/en/release-notes/serverless.html."
 2146 )
 2147 # END-EDGE
-> 2149 raise convert_exception(
 2150 info,
 2151 status.message,
 2152 self._fetch_enriched_error(info),
 2153 self._display_server_stack_trace(),
 2154 ) from None
 2156 raise SparkConnectGrpcException(status.message) from None
 2157 else:

AnalysisException: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 7690b6f5-c514-4e4c-96e7-bc881bfece79).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)
-- salary: long (nullable = true)
-- name: string (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- salary: long (nullable = true)
-- name: string (nullable = true)
-- sus_col: long (nullable = true)

 
Table ACLs are enabled in this cluster, so automatic schema migration is not allowed. Please use the ALTER TABLE command for changing the schema. 

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at

# **SCHEMA EVOLUTION**

***when schema is different or chamged use (mergeschema)***

In [0]:
df.write.format("delta")\
        .mode("append")\
        .option("mergeSchema",True)\
        .save("/Volumes/deltalakevinay/default/deltavol/demosink/")

# READ DELTA DATA

### **TABLE**

In [0]:
%sql
SELECT * FROM deltalakevinay.default.firstdeltaapi

salaryAfterTax,salary,name
70,100,aa
140,200,bb


### **DATA LAKE**

In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakevinay/default/deltavol/demosink/`

id,salary,name,sus_col
1,100,aa,1
2,200,bb,1
3,300,cc,1
4,400,dd,1
1,100,aa,null
2,200,bb,null
3,300,cc,null
4,400,dd,null
1,100,aa,null
2,200,bb,null


# **SCHEMA OVERWRITE**

In [0]:
data = [(1,100,'aa',1),(2,200,'bb',1),(3,300,'cc',1),(4,400,'dd',1)]

df = spark.createDataFrame(data, ['cust_id','income','name','tip'])

display(df)

cust_id,income,name,tip
1,100,aa,1
2,200,bb,1
3,300,cc,1
4,400,dd,1


In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("overwriteSchema",True)\
        .save("/Volumes/deltalakevinay/default/deltavol/demosink/")